In [ ]:
!pip install langchain openai
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.6 MB/s eta 0:00:00


In [ ]:
!pip install langchain openai deep-translator


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain openai googletrans

from langchain.agents import Tool, AgentExecutor, ZeroShotAgent, Agent
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain # Import LLMChain
from googletrans import Translator
from deep_translator import GoogleTranslator

def translate_text(text, target_lang):
    # Use GoogleTranslator from deep-translator
    translator = GoogleTranslator(source='auto', target=target_lang)
    translation = translator.translate(text)
    return translation

# Define the translation tool
tools = [
    Tool(
        name="Translator",
        func=lambda x: translate_text(*x.split(";")),  # Split input and pass as arguments
        description="Useful for translating text to different languages. Input should be a text string and the desired target language in the format 'Translate <text> to <language_code>'. For example, 'Translate Hello to es' translates 'Hello' to Spanish.",
    )
]

# Initialize the language model (replace with your actual OpenAI API key)
llm = OpenAI(temperature=0, openai_api_key="Your Openai Key")

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["input"],
    template="""You are a helpful AI assistant that can translate text. You have access to the following tool:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the name of the tool you use
Action Input: the input to the tool, should be text and target language separated by ;
Observation: the output of the tool
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
""",
)

# Create the agent, passing in the LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Create a ZeroShotAgent instance
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

# Use AgentExecutor to manage the agent's execution
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Update the input to match the expected format
response = agent_executor.invoke({"input": "Translate School;hindi", "tools": tools})
print(response)



> Entering new AgentExecutor chain...
Thought: I should use the Translator tool to translate the word "School" to Hindi.
Action: Translator
Action Input: School;hindi
Observation: विद्यालय
Thought: I should use the Translator tool to translate the word "विद्यालय" back to English to confirm the translation.
Action: Translator
Action Input: विद्यालय;english
Observation: the school
Thought: I now know the final answer.
Final Answer: The translation of "School" to Hindi is विद्यालय.

> Finished chain.
{'input': 'Translate School;hindi', 'tools': [Tool(name='Translator', description="Useful for translating text to different languages. Input should be a text string and the desired target language in the format 'Translate <text> to <language_code>'. For example, 'Translate Hello to es' translates 'Hello' to Spanish.", func=<function <lambda> at 0x78de7c87d120>)], 'output': 'The translation of "School" to Hindi is विद्यालय.'}
